<a href="https://colab.research.google.com/github/carlosinator/cil-sentiment/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%%bash
pip3 install transformers emoji==0.6.0 keras_nlp

In [3]:
!git clone https://github.com/carlosinator/cil-sentiment.git

fatal: destination path 'cil-sentiment' already exists and is not an empty directory.


In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras_nlp
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
import transformers
from transformers import AutoTokenizer, TFAutoModel, AutoConfig, TFAutoModelForSequenceClassification
import matplotlib.pyplot as plt
import pickle
import re
import subprocess as sp
import os
from threading import Thread , Timer
import sched, time

import sys
sys.path.append("./cil-sentiment/models")
sys.path.append("./cil-sentiment/")
from gru_models import GRUModel, VGRUModel
import utils

# reproducibility
transformers.set_seed(0) # sets the seed in random, numpy, and tf

Using TensorFlow backend


In [5]:
!gsutil cp "gs://cil_2023/train_pos_full_preprocessed_without_duplicates.txt" .
!gsutil cp "gs://cil_2023/train_neg_full_preprocessed_without_duplicates.txt" .

model_name = "vinai/bertweet-base"
filename_train_pos = "train_pos_full_preprocessed_without_duplicates.txt"
filename_train_neg = "train_neg_full_preprocessed_without_duplicates.txt"

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

Copying gs://cil_2023/train_pos_full_preprocessed_without_duplicates.txt...
| [1 files][ 74.6 MiB/ 74.6 MiB]                                                
Operation completed over 1 objects/74.6 MiB.                                     
Copying gs://cil_2023/train_neg_full_preprocessed_without_duplicates.txt...
/ [1 files][ 91.6 MiB/ 91.6 MiB]                                                
Operation completed over 1 objects/91.6 MiB.                                     


In [6]:
# read dataset
dataset_pos_pd = pd.read_table(filename_train_pos, sep='\r\n', header=None, names=['text'])
dataset_neg_pd = pd.read_table(filename_train_neg, sep='\r\n', header=None, names=['text'])
dataset_pos_pd['label'] = 0
dataset_neg_pd['label'] = 1
dataset_pd = pd.concat([dataset_pos_pd, dataset_neg_pd])

# shuffle
dataset_pd = dataset_pd.sample(frac=1, random_state=0).reset_index(drop=True)

# tokenize data set
tokenizer = AutoTokenizer.from_pretrained(model_name)
texts = tokenizer.batch_encode_plus(dataset_pd['text'].tolist(),
                                    padding=True, truncation=True,
                                    return_tensors='tf')

dataset = tf.data.Dataset.from_tensor_slices((dict(texts), dataset_pd['label']))

# split training / validation
batch_size = 256 # * tpu_strategy.num_replicas_in_sync
AUTOTUNE = tf.data.experimental.AUTOTUNE

val_data_size = int(0.1 * len(dataset_pd.index))
test_data_size = int(0.1 * len(dataset_pd.index))
train_data_size = len(dataset_pd.index) - val_data_size - test_data_size
val_ds = dataset.take(val_data_size).batch(batch_size, drop_remainder=True)
test_ds = dataset.take(test_data_size).batch(batch_size, drop_remainder=True)
train_ds = dataset.skip(val_data_size).batch(batch_size, drop_remainder=True)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

<ipython-input-6-a25b9d79ca72>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset_pos_pd = pd.read_table(filename_train_pos, sep='\r\n', header=None, names=['text'])
<ipython-input-6-a25b9d79ca72>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset_neg_pd = pd.read_table(filename_train_neg, sep='\r\n', header=None, names=['text'])
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Train model for inference

In [7]:
import subprocess as sp
import os
from threading import Thread , Timer
import sched, time

gpu_hist = {
    "mib" : [],
    "percent" : [],
    "counter" : 0,
}

def get_gpu_memory():

    out = !nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv

    float_pattern = r'\d+\.\d+|\d+'
    numbers = re.findall(float_pattern, out[1])

    # Extract the MiB value and the percentage
    mib_val = float(numbers[0])
    perc = float(numbers[1])

    gpu_hist["counter"] += 1

    # print(mib_val, perc)

    gpu_hist["mib"].append(mib_val)
    gpu_hist["percent"].append(perc)

    return


def track_gpu_mem(interval=5.0, dictname="gpu_hist.pkl"):
    """
        This function calls itself every 5 secs and print the gpu_memory.
    """
    thd = Timer(interval, track_gpu_mem, [interval, dictname])
    thd.start()
    get_gpu_memory()

    gpu_hist["interval"] = interval

    with open(dictname, 'wb') as f:
      pickle.dump(gpu_hist, f)

    return thd

In [8]:
USE_MODEL = "basemodel"
LEARNING_RATE = 2e-5
EPOCHS = 10
model = utils.get_model(model_name, LEARNING_RATE, USE_MODEL)
history = {}

run_name = "inference_" + USE_MODEL + "_fullmodel"
hd_name = run_name + "_dict.pkl"
gpu_hist_name = "gpu_hist_" + USE_MODEL + ".pkl"

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
tf.keras.backend.clear_session()

In [10]:
callback = tf.keras.callbacks.EarlyStopping(patience=3) # early stopping def
gpu_mem_proc = track_gpu_mem(10.0, dictname=gpu_hist_name) # initialize gpu tracking

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[callback]
    )

gpu_mem_proc.join()

Epoch 1/10
7981/7981 [==============================] - 3007s 371ms/step - loss: 0.2373 - sparse_categorical_accuracy: 0.8998 - val_loss: 0.2188 - val_sparse_categorical_accuracy: 0.9087
Epoch 2/10
3271/7981 [===========>..................] - ETA: 27:37 - loss: 0.2061 - sparse_categorical_accuracy: 0.9143

7981/7981 [==============================] - 2906s 364ms/step - loss: 0.2016 - sparse_categorical_accuracy: 0.9165 - val_loss: 0.2237 - val_sparse_categorical_accuracy: 0.9095
Epoch 3/10
7981/7981 [==============================] - 2907s 364ms/step - loss: 0.1742 - sparse_categorical_accuracy: 0.9292 - val_loss: 0.2346 - val_sparse_categorical_accuracy: 0.9080
Epoch 4/10
7981/7981 [==============================] - 2906s 364ms/step - loss: 0.1492 - sparse_categorical_accuracy: 0.9406 - val_loss: 0.2540 - val_sparse_categorical_accuracy: 0.9056


In [15]:
# save model
model.save(run_name)
# save training history
with open(hd_name, 'wb') as f:
  pickle.dump(history, f)

#Note: the gpu history should already been stored at gpu_hist_name

!gsutil cp -r {run_name + "/"} "gs://cil_2023/models/"
!gsutil cp {hd_name} "gs://cil_2023/models/"
!gsutil cp {gpu_hist_name} "gs://cil_2023/models/"

Copying file://inference_basemodel_fullmodel/fingerprint.pb [Content-Type=application/octet-stream]...
Copying file://inference_basemodel_fullmodel/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://inference_basemodel_fullmodel/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://inference_basemodel_fullmodel/variables/variables.index [Content-Type=application/octet-stream]...
\
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://inference_basemodel_fullmodel/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_compo

# Retrieve model for inference

In [27]:
# uncomment if trained_model is not on local machine
# !gsutil cp -r {"gs://cil_2023/models/" + run_name} .
trained_model = tf.keras.models.load_model(run_name)
trained_model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  134309376 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  592130    
 ificationHead)                                                  
                                                                 
Total params: 134901506 (514.61 MB)
Trainable params: 134901506 (514.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Test accuracy

In [ ]:
preds = trained_model.predict(test_ds)

if USE_MODEL == "basemodel":
  preds = tf.keras.layers.Softmax()(preds)